In [38]:
import pandas as pd
from tqdm.auto import tqdm
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\cri98\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [10]:
df = pd.read_csv("selected_patents.csv").drop_duplicates()

df.head()

,filename,filing_date,publication_date,priority_date,ipc_classes,assignee,inventors,docdb_family_id,title,abstract,claims
0,US10849406_B2.txt,2015-12-01,2020-12-01,2014-12-02,"A45D40/00, A45D40/26, A45D44/00, B01F13/10, B0...",L'OREAL,"GIRON, FRANCKSAMAIN, HENRI",54780284,system for dispensing at least one makeup prod...,a system for dispensing at least one makeup pr...,1. a system for dispensing at least one makeup...
1,WO2020176430_A1.txt,2020-02-24,2020-09-03,2019-02-26,"A45D29/00, A45D29/11, A45D29/12, A45D29/14, A4...",ELEMENTREE,"SUNDEN, ERIKAPTE, RENUKAFELDSTEIN, AARON",72142514,automatic nail polish application system and m...,the present disclosure relates to a robotic ap...,1. a system for automatically polishing a nail...
2,WO2019122838_A1.txt,2018-12-18,2019-06-27,2017-12-22,"A45D1/00, G09B19/00, G09B19/24, G09B5/04",JEMELLA,"JAMES, MATTMOORE, TIMOTHYRAMOS, DANIEL",61131465,training system and device,a beauty product device is disclosed which inc...,1. a beauty product system for performing a tr...
3,US7458944_B2.txt,2004-04-05,2008-12-02,2001-10-04,"A45D34/04, A61H15/02, A61H7/00, A61K8/00, A61K...",BEIERSDORF,"LISTE, KATHRINRHODE, OLAFTREU, JENS",7701353,cosmetic cleaning product,"the invention is a cosmetic cleaning product, ...","1. a cosmetic cleaning product, comprising: a ..."
4,US10820678_B2.txt,2017-03-27,2020-11-03,2016-03-25,"A45D20/12, A45D20/14, F16M11/00, F16M13/02","KARAMBATSOS, NATASA","KARAMBATSOS, NATASA",59896261,portable apparatus for supporting a hand held ...,a portable hair dryer support apparatus compri...,"1. a portable hair dryer support apparatus, co..."


In [31]:
columns = df.columns.to_list()
tokeep = [True for _ in range(len(df))]

df_sorted = df.sort_values(by=["docdb_family_id", "claims"])

pbar = tqdm(zip(df_sorted.values[1:], df_sorted.values[:-1]), total=len(df)-1)

for i, (row1, row2) in enumerate(pbar):
    pbar.set_description(f'{row1[columns.index("docdb_family_id")]} vs {row2[columns.index("docdb_family_id")]}')
    
    if row1[columns.index("docdb_family_id")] != row2[columns.index("docdb_family_id")]:
        continue
    
    #dist = nltk.edit_distance(row1[columns.index("claims")], row2[columns.index("claims")])
    dist = nltk.jaccard_distance(row1[columns.index("claims")], row2[columns.index("claims")])
    print(f'({row1[columns.index("docdb_family_id")]})edit distance: {dist}')
    if dist > 100:
        tokeep[i] &= False

  0%|          | 0/38 [00:00<?, ?it/s]

7701353()edit distance: 924
35580235()edit distance: 3781
35580235()edit distance: 4759
35580235()edit distance: 6091
59896261()edit distance: 4175
62838170()edit distance: 6410
63798904()edit distance: 83
65279789()edit distance: 1364
67067677()edit distance: 315


In [40]:
columns = df.columns.to_list()
tokeep = [True for _ in range(len(df))]

df_sorted = df.sort_values(by=["docdb_family_id", "claims"])

pbar = tqdm(zip(df_sorted.values[1:], df_sorted.values[:-1]), total=len(df)-1)

for i, (row1, row2) in enumerate(pbar):
    pbar.set_description(f'{row1[columns.index("docdb_family_id")]} vs {row2[columns.index("docdb_family_id")]}')
    
    if row1[columns.index("docdb_family_id")] != row2[columns.index("docdb_family_id")]:
        continue
    
    #dist = nltk.edit_distance(row1[columns.index("claims")], row2[columns.index("claims")])
    dist = nltk.jaccard_distance(set(nltk.word_tokenize(row1[columns.index("claims")])), 
                                 set(nltk.word_tokenize(row2[columns.index("claims")])))
    print(f'({row1[columns.index("docdb_family_id")]})jaccard distance: {dist}')
    if dist > .1:
        tokeep[i] &= False

  0%|          | 0/38 [00:00<?, ?it/s]

(7701353)jaccard distance: 0.10795454545454546
(35580235)jaccard distance: 0.24363636363636362
(35580235)jaccard distance: 0.35962145110410093
(35580235)jaccard distance: 0.5133333333333333
(59896261)jaccard distance: 0.4147727272727273
(62838170)jaccard distance: 0.478494623655914
(63798904)jaccard distance: 0.053811659192825115
(65279789)jaccard distance: 0.3918918918918919
(67067677)jaccard distance: 0.0137524557956778


In [19]:
columns.to_list().index("docdb_family_id")

7